## Listening to samples from the DMM
After running the shell scripts in `expt`, we can listen to samples from the model.

In [11]:
import os
import numpy as np
from utils.misc import loadHDF5, createIfAbsent
assert os.path.exists('../expt/chkpt-jsb/'),'Run the shell files in ../expt first'
import glob, os, sys, time
sys.path.append('../')
from utils.misc import getConfigFile, readPickle, displayTime, downloadData, loadHDF5, getLowestError
start_time = time.time()
from   model_th.dmm import DMM
import model_th.learning as DMM_learn
import model_th.evaluate as DMM_evaluate
displayTime('importing DMM',start_time, time.time())

		< importing DMM > took  0.000238180160522   seconds


We need to download the midi library available [here](http://www.iro.umontreal.ca/~lisa/deep/midi.zip) to the current folder. 

In [3]:
if not os.path.exists('./midi/'):
    download_dir = './'
    files        = {}
    files['midi.zip'] = 'http://www.iro.umontreal.ca/~lisa/deep/midi.zip'
    downloadData(download_dir,files)
    os.system('unzip midi.zip;rm -rf midi.zip')
else:
    print './midi found'

./midi found


In [5]:
ls midi/utils*

midi/utils.py  midi/utils.pyc


We will also need the timidity package to convert from midi to wav formats

In [7]:
from midi.utils import midiread, midiwrite
assert os.system('timidity -h')==0,'Install Timidity from http://timidity.sourceforge.net/'

### Reload DMM

In [24]:
DATASET= 'jsb'
DIR    = '../expt/chkpt-'+DATASET+'/'
prefix = 'DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar'
stats  = loadHDF5(os.path.join(DIR,prefix+'-final.h5'))
epochMin, valMin, idxMin = getLowestError(stats['valid_bound'])
pfile  =  os.path.join(DIR,prefix+'-config.pkl')

params = readPickle(pfile, quiet=True)[0]
print 'Hyperparameters in: ',pfile, 'Found: ',os.path.exists(pfile)
EP     = '-EP'+str(int(epochMin))
reloadFile  =  os.path.join(DIR,prefix+EP+'-params.npz')
print 'Model parameters in: ',reloadFile
#Don't load the training functions for the model since its time consuming
params['validate_only'] = True
dmm_reloaded  = DMM(params, paramFile = pfile, reloadFile = reloadFile)

Hyperparameters in:  ../expt/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-config.pkl Found:  True
Model parameters in:  ../expt/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP890-params.npz
	<<Reloading Model>>
	<<Loading structure (../expt/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-config.pkl) and model (../expt/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP890-params.npz) / opt (../expt/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP890-optParams.npz) weights>>
Adding/Modifying loaded parameters:  validate_only  to  True
	<<Nparameters: 3295291>>
	<<Anneal = 1 in 2000.

In [25]:
params, zvec = DMM_evaluate.sample(dmm_reloaded, T= 200)
bin_prob     = params[0]
print 'Samples: ',bin_prob.shape

Samples:  (100, 200, 88)


In [31]:
#Parameters for music sampling taken from http://deeplearning.net/tutorial/rnnrbm.html
rval = (21, 109)
dt   = 0.3
SAVEDIR = './samples/'
createIfAbsent(SAVEDIR)
print 'Saving wav...'
for idx, sample in enumerate(bin_prob):
    piano_roll = (sample>0.5)*1.
    filename = SAVEDIR+DATASET+'-'+str(idx)+'.midimport IPython
IPython.display.Audio("my_audio_file.mp3")'
    midiwrite(filename, piano_roll, rval, dt)
    print idx,

print 'Converting...'
print os.system('cd '+SAVEDIR+';timidity -Ow1S *.mid;cd ../')
print '\nFiles\n',', '.join(os.listdir(SAVEDIR))

Saving wav...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 Converting...
0


In [35]:
import IPython
IPython.display.Audio('./samples/jsb-0.wav')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
